In [ ]:
# Import packages
import numpy as np
import pandas as pd
import scipy
from sklearn.metrics import mutual_info_score

In [ ]:
# Import single-cell counts and cluster labels
sct_data = pd.read_csv("") # Path to counts csv
labels = pd.read_csv("") # Path to cluster labels csv

In [ ]:
sct_data_dropped = sct_data.drop(["Unnamed: 0"], axis = 1)
labels.set_index("Unnamed: 0", inplace = True)

In [ ]:
# Flatten the data
sct_data_flat = sct_data_dropped.values.flatten()

In [ ]:
# Binarize expression values and format for calculating mutual information
bins = [0, 1, np.max(sct_data_flat)]
expression_labels = ["Off","On"]
sct_discrete = pd.cut(sct_data_flat, bins = bins, labels = expression_labels, include_lowest = True)
sct_discrete = pd.DataFrame(np.array(sct_discrete).reshape(sct_data_dropped.shape))
sct_discrete.columns = sct_data_dropped.columns
sct_discrete["Genes"] = sct_data[["Unnamed: 0"]]
sct_discrete.set_index("Genes", inplace=True)

In [ ]:
def calc_mutual_information(df_discrete, labels):
    mutual_informations = []
    for symbol, row in df_discrete.iterrows():
        mi = mutual_info_score(row, labels) / np.log(2) # calculate mutual information, convert from nats to bits
        mutual_informations.append(mi)
    df_result = pd.DataFrame()
    df_result["symbol"] = df_discrete.index
    df_result.set_index("symbol", inplace=True)
    df_result["mutual_information"] = mutual_informations
    df_result.sort_values("mutual_information", inplace=True, ascending=False)
    return df_result

In [ ]:
mutual_info = calc_mutual_information(sct_discrete, labels["x"])

In [ ]:
pd.DataFrame.to_csv(mutual_info, "mutual_information.csv")